In [1]:
from ase.calculators.lammpsrun import LAMMPS
from ase.io import read

import mcmc
import os

final_cifs = [
    "/home/dux/surface_sampling/sgmc_Cu_surf/GaN_0001_3x3/runs300_temp1_adsatoms12_alpha0.99_20220427-1930/optim_slab_run_300.cif",
    "/home/dux/surface_sampling/sgmc_Cu_surf/GaN_0001_3x3/runs300_temp1_adsatoms12_alpha0.99_20220427-2157/optim_slab_run_300.cif",
    "/home/dux/surface_sampling/sgmc_Cu_surf/GaN_0001_3x3/runs300_temp1_adsatoms12_alpha0.99_20220428-1008/optim_slab_run_300.cif",
    "/home/dux/surface_sampling/sgmc_Cu_surf/GaN_0001_3x3/runs300_temp1_adsatoms12_alpha0.99_20220428-1014/optim_slab_run_300.cif"
]

slabs = [read(cif) for cif in final_cifs]

# potential
parameters = {
    'pair_style': 'tersoff',
    'pair_coeff': ['* * GaN.tersoff Ga N']
}
potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'GaN.sw')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=False, tmp_dir="/home/dux/surface_sampling/tmp_files")
lammps_calc.set(**parameters)

# set calc
for slab in slabs:
    slab.calc = lammps_calc

energies = [slab.get_potential_energy() for slab in slabs]

print(f"energies = {energies}")

energies = [-179.4118780753384, -179.5810961988868, -179.4651218516284, -179.6430142516041]


In [4]:
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = read("GaN_0001_3x3_pristine_slab.cif")
num_bulk_atoms = len(pristine_slab)

# just the adsorbates
lit_slab = read("GaN_0001_3x3_12_Ga_ads_arranged_slab.cif")[num_bulk_atoms:]
lit_optim_slab = read("GaN_0001_3x3_12_Ga_ads_original_optim_arranged_slab.cif")[num_bulk_atoms:]

lit_pmg = AseAtomsAdaptor.get_structure(lit_slab)
lit_optim_pmg = AseAtomsAdaptor.get_structure(lit_optim_slab)
expt_optim_pmgs = [AseAtomsAdaptor.get_structure(slab)[num_bulk_atoms:] for slab in slabs]

struc_matcher = StructureMatcher(primitive_cell=False)
'''
Returns:
    rms displacement normalized by (Vol / nsites) ** (1/3) and maximum distance between paired sites.
    If no matching lattice is found None is returned.
'''
print(f"distance btw lit optim and lit    {struc_matcher.get_rms_dist(lit_optim_pmg, lit_pmg)}")
for i, pmg in enumerate(expt_optim_pmgs):
    print(f"distance btw lit optim and expt {i+1} {struc_matcher.get_rms_dist(lit_optim_pmg, pmg)}")

distance btw lit optim and lit    (0.01287754772691926, 0.02094690414206456)
distance btw lit optim and expt 1 (0.13092956831157618, 0.2168180678039432)
distance btw lit optim and expt 2 (0.17833628843868046, 0.2549708663260898)
distance btw lit optim and expt 3 (0.07925401710348659, 0.17252930728551777)
distance btw lit optim and expt 4 (0.18830011333083602, 0.25013559902101234)
